# 자연어처리 시작하기
* soynlp(https://github.com/lovit/soynlp)로 토큰화하고 명사를 추출해 워드클라우드 그려보기

In [ ]:
# !pip install soynlp

In [ ]:
!pip show soynlp

In [ ]:
import pandas as pd
import numpy as np
import re

# 데이터 로드하기
*  데이터 출처 : https://github.com/akngs/petitions

In [ ]:
import os
import platform

base_path = "data"
file_name = "petition.csv"
url ='https://drive.google.com/open?id=1d8VEAj6n83wT1YRlCrhoU_1mMSvYpXc4'

def file_exist_check(base_path, file_name):
    if os.path.exists(f"./{file_name}"):
        print(f"{os.getcwd()}/{base_path} 경로에 파일이 있음")
        return

    if not os.path.exists(base_path):
        os.makedirs(base_path)

    if platform.system() == "Linux":
        # 구글 드라이브에서 csv 파일을 읽어오기 위해 gauth 인증하기
        !pip install -U -q PyDrive
        from pydrive.auth import GoogleAuth
        from pydrive.drive import GoogleDrive
        from google.colab import auth
        from oauth2client.client import GoogleCredentials
        
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)

        id = url.split('=')[1]
        downloaded = drive.CreateFile({'id':id}) 
        downloaded.GetContentFile(f'{base_path}/{file_name}')  
        print(f"사용자의 구글 드라이브에 {base_path}/{file_name} 다운로드 완료")
    else:
        print(f"{url} 에서 다운로드 받아 실습 경로 {os.getcwd()}/{base_path}에 옮겨주세요.""")
        return
    
file_exist_check(base_path, file_name) 

In [ ]:
df = pd.read_csv(f"{base_path}/petition.csv", index_col="article_id",
                        parse_dates=['start', 'end'])
df.shape

In [ ]:
df.shape

In [ ]:
df.tail()

## 자신의 관심사에 맞는 단어로 데이터를 가져옵니다.

In [ ]:
p = r'.*(돌봄|육아|초등|보육).*'
care = df[df['title'].str.match(p) |
           df['content'].str.match(p, flags=re.MULTILINE)]
care.shape

In [ ]:
care.head()

In [ ]:
care.tail(3)

In [ ]:
# 샘플로 보고 싶은 인덱스의 번호를 넣어주세요.
sample_index = 24

In [ ]:
sample_title = care.loc[sample_index, 'title']
sample_title

In [ ]:
sample_content = care['content'][sample_index]
sample_content

In [ ]:
content_text = care['content'].str.replace("\\\\n", " ", regex=True)
content_text = content_text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 a-zA-Z]", " ", regex=True)

In [ ]:
content_text.head(2)

# 토큰화

In [ ]:
from soynlp.tokenizer import RegexTokenizer

tokenizer = RegexTokenizer()
tokenizer

In [ ]:
tokened_title = tokenizer.tokenize(sample_title)
tokened_title

In [ ]:
tokened_content = tokenizer.tokenize(sample_content)
tokened_content[:20]

In [ ]:
print(len(tokened_title))
print(len(tokened_content))

# 텍스트 데이터 전처리 
* 개행문자 제거

In [ ]:
content_text

In [ ]:
%time tokens = content_text.apply(tokenizer.tokenize)
tokens[:3]

In [ ]:
tokens[sample_index][:10]

In [ ]:
# !pip install wordcloud
# !pip install koreanize-matplotlib

In [ ]:
import koreanize_matplotlib
# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 폰트 설정시 폰트명이 아닌 폰트의 설치 경로를 입력해 주셔야 합니다.
# 윈도우 : r"C:\Windows\Fonts\malgun.ttf" 해당 경로에 폰트가 있는지 확인한다.
# 맥 : r"/Library/Fonts/AppleGothic.ttf"
# colab에서 나눔고딕 등의 폰트를 설치했다면 : '/Library/Fonts/NanumBarunGothic.ttf'
# fontpath=r"/Library/Fonts/NanumBarunGothic.ttf",

from wordcloud import WordCloud
import matplotlib.pyplot as plt


def display_word_cloud(data, width=1200, height=500):
    
    stopwords = ['하지만', '그리고', '그런데', '저는','제가',
                '그럼', '이런', '저런', '합니다',
                '많은', '많이', '정말', '너무'] 
    
    word_draw = WordCloud(
        font_path=r"/Library/Fonts/NanumBarunGothic.ttf",
        width=width, height=height,
        stopwords=stopwords, 
        background_color="white",
        random_state=42
    )
    word_draw.generate(data)

    plt.figure(figsize=(15, 7))
    plt.imshow(word_draw)
    plt.axis("off")
    plt.show()

In [ ]:
# 결과를 출력해 보면 불용어(STOPWORD)가 너무 많습니다.
%time display_word_cloud(' '.join(content_text))

In [ ]:
from soynlp.noun import LRNounExtractor

In [ ]:
%%time
noun_extractor = LRNounExtractor(verbose=True)
noun_extractor.train(content_text)
nouns = noun_extractor.extract()

In [ ]:
nouns_text = " ".join(list(nouns.keys()))
nouns_text[:100]

In [ ]:
type(nouns_text)

In [ ]:
# 추출된 명사를 찍어봅니다.
%time display_word_cloud(nouns_text)